# Depth Estimator Model

### Dependencies

In [44]:
import cv2
from PIL import Image
import numpy as np
import tensorflow as tf
import xgboost as xg
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input